# Data cleaning
---

This script takes .cvs as input, and it cleans the data frame by
- Removing unwanted feature
- Convert long string into multiple dummy variables
- Filling missing values
- Extract numerical features from string

### Import libraries and data

In [74]:
import pandas as pd
import numpy as np
import math, time, os
#import googlemaps
#import geocoder

%matplotlib inline

In [75]:
# read data from csv
df = pd.read_csv("data/ny_rental_data_v2.csv", index_col = 0)

In [76]:
df.shape

(7797, 15)

### Missing value analysis

In [77]:
na_cols, na_counts = [], []
for col in df.columns[:-1]:
    if(df[col].isnull().any()):
        na_cols.append(col)
        na_counts.append(len(df[df[col]!=df[col]]))
        print('({})  {}, {} NA values'.format(df[col].dtype, col, len(df[df[col]!=df[col]])))
print('\nOut of total {} columns, there are {} columns containing NA values'.format(len(df.columns[:-1]), len(na_cols)))

(object)  rating, 2649 NA values
(object)  property_info, 2559 NA values
(object)  bathrooms, 791 NA values
(object)  rent, 1073 NA values

Out of total 14 columns, there are 4 columns containing NA values


In [78]:
# select digits and return
def digit_filter(string):

    digit = ''.join(filter(lambda x: x.isdigit(), str(string)))

    if digit == '':
        return 999
    else:
        return int(digit)

### Rent

In [79]:
# find the mean rent of a property
def find_mean(string):

    # temporary list
    list_temp = [digit_filter(x) for x in str(string).split()]

    # return mean of the temp
    if not list_temp[0] == 0:

        # remove 0 from list
        list_temp = [x for x in list_temp if not x == 0]

        # return mean
        return round(sum(list_temp) / len(list_temp),2)

    # if list_temp is empty, return 0
    else:
        return 0

In [80]:
# drop rows without rent
df.dropna(subset=["rent"], inplace=True)

In [81]:
# apply find_mean
df["rent"] = df["rent"].apply(find_mean)

In [82]:
df.rent.describe()

count     6724.000000
mean      2538.930973
std       1658.584654
min        600.000000
25%       1475.000000
50%       2225.000000
75%       3150.000000
max      57000.000000
Name: rent, dtype: float64

### Bathrooms

In [83]:
df["bathrooms"].value_counts()

1 BA      5041
2 BAs     1331
1½ BAs     170
2½ BAs      79
3 BAs       69
3½ BAs      18
4 BAs        2
0½ BAs       2
5 BAs        1
6 BAs        1
Name: bathrooms, dtype: int64

In [84]:
df.bathrooms.isnull().sum()

10

In [85]:
def bathrooms_clean(bathrooms):
    # note: at most 6 bathrooms
    
    baths = bathrooms.split()[0]
    
    if baths.isdigit():
        return int(baths)
    else:                # for the case of half bathrooms, e.g. 1½ BAs 
        return int(baths[:-1]) + 0.5

In [86]:
# we are replacing NA with 999 BA's

df["bathrooms"] = df["bathrooms"].fillna("999 BAs").apply(bathrooms_clean)

In [87]:
df["bathrooms"].value_counts()

1.0      5041
2.0      1331
1.5       170
2.5        79
3.0        69
3.5        18
999.0      10
4.0         2
0.5         2
5.0         1
6.0         1
Name: bathrooms, dtype: int64

### Bedrooms

In [88]:
df["bedrooms"].value_counts()

1 BR      2603
2 BRs     2024
Studio    1007
3 BRs      897
4 BRs      164
5 BRs       20
6 BRs        8
7 BRs        1
Name: bedrooms, dtype: int64

In [89]:
def bedrooms_clean(bedrooms):
    # note: at most 7 bedrooms, including studio
    
    beds = bedrooms.split()[0]
    
    if beds.isdigit():
        return int(beds)
    else:                   # for the case of studio
        return 0

In [90]:
df["bedrooms"] = df["bedrooms"].fillna("999 BRs").apply(bedrooms_clean)

In [91]:
df["bedrooms"].value_counts()

1    2603
2    2024
0    1007
3     897
4     164
5      20
6       8
7       1
Name: bedrooms, dtype: int64

### Property info:  Extract built year and number of units

In [92]:
def built_year(string):
    # extract year
    year = [digit_filter(x) for x in str(string).split('\n') if "built in" in x.lower()]

    # return valid year
    if year:
        return year[0]
    else:
        return 999

In [93]:
def apartment_age(year):
    return pd.datetime.now().year - year

In [94]:
def property_size(string):
    # extract property size
    string = str(string).replace('/','\n')

    # extract units
    units = [digit_filter(x) for x in str(string).split('\n') if "units" in x.lower()]

    # return valid units
    if units:
        return units[0]
    else:
        return 999

In [95]:
# create new column containing built year
df["built_year"] = df["property_info"].apply(built_year)

# apartment age
df["apartment_age"] = df["built_year"].apply(apartment_age)

# create new column containing property size
df["property_size"] = df["property_info"].apply(property_size)

# fill null value with the median property_size # mean
#df["property_size"].fillna(df["property_size"].median(), inplace=True)

# convert float to int
#df["property_size"] = df["property_size"].astype(int)

In [96]:
# remove property_info after extracting desired info
df.drop("property_info", axis=1, inplace=True)

### Property rating

In [97]:
# extract rating number
df["rating"] = df["rating"].apply(digit_filter)

# replace no rating with Nan
#df["rating"] = df["rating"].apply(lambda x: None if x == 0 else x)

# replace Nan with average            (# this is only temporary)
#df["rating"].fillna(df["rating"].mean(), inplace=True)

# convert to int
#df["rating"] = df["rating"].astype(int)

In [98]:
df.rating.value_counts()

5      3304
999    2636
4       565
3       151
2        38
1        30
Name: rating, dtype: int64

### Sqft

In [99]:
def sqrt_clean(sqrt):
    return int(''.join(sqrt.split()[0].split(',')))

In [100]:
# apply "find_mean" to sqft column
df["sqft"] = df["sqft"].fillna('0 Sq Ft').apply(sqrt_clean)

In [101]:
# set limits of sqft
sqft_factor_min = 0.5
sqft_min = 100

In [102]:
# replace un-realistic sqft with None
df.loc[(df["sqft"] / ((df["bedrooms"] + 1) * df["rent"])) > sqft_factor_min, "sqft"] = 0 # sqft too large
df.loc[df.sqft < sqft_min, "sqft"] = 0 # sqft too small

# fill null with average sqft of apartments with the same number of bedrooms
#df["sqft"] = df.groupby(["bedrooms"])["sqft"].transform(lambda x: x.fillna(x.mean()))

### Borough

In [103]:
# clean up and capitalize
df["borough"] = df.borough.transform(lambda x: x[:-3].replace('-',' ').title())

### Pet policy

In [104]:
# function to apply to pet_policy column
def std_pet_policy(pet):
    # list to temporary hold std pet
    std_pet_list = []

    # remove un-wanted characters
    pet = pet.replace('[','')
    pet = pet.replace(']','')
    pet = pet.replace("'","")

    # create list of pet from the string
    pet_list = pet.split(",")

    # for each pet found in the unit, find std pet from the dictionary
    for std_pet in dic_pet:
        if any(std_pet in pet.lower() for pet in pet_list):
            std_pet_list.append(dic_pet[std_pet])

    # return std pet
    return std_pet_list

In [105]:
# define list of pets to check
with open('data/pet','r') as dic:
    dic_pet = eval(dic.read())

# apply "std_pet_policy"
df["pet_policy"] = df["pet_policy"].apply(std_pet_policy)

In [106]:
df.pet_policy.value_counts()

[dog, cat]    4337
[]            2034
[cat]          239
[dog]          114
Name: pet_policy, dtype: int64

In [107]:
# concatenate dummy DataFrame
df_dummy = df["pet_policy"].str.join(sep='*').str.get_dummies(sep='*')

# add prefix to all pet dummy columns
df_dummy = df_dummy.add_prefix("pet_")

# concatenate dummy to the main DataFrame
df = pd.concat([df, df_dummy], axis=1)

# remove pet column since we converted it to dummies
df.drop("pet_policy", axis=1, inplace=True)

### Amenities

In [108]:
# function to apply to amenity column
def std_amenity(amenity):
    # list to temporary hold std amenties
    std_amenity_list = []

    # remove un-wanted characters
    amenity = amenity.replace('[','')
    amenity = amenity.replace(']','')
    amenity = amenity.replace("'","")

    # create list of amenties from the string
    amenity_list = amenity.split(",")

    # for each amenity found in the unit, find std amenity from the dictionary
    for sam in dic_amenity:
        if any(sam in am.lower() for am in amenity_list):
            std_amenity_list.append(dic_amenity[sam])

    # return std amenity
    return std_amenity_list

In [109]:
# define list of amenties to check
with open('data/amenities','r') as dic:
    dic_amenity = eval(dic.read())

# apply "std_amenity"
df["amenity"] = df["amenity"].apply(std_amenity)

# concatenate dummy DataFrame
df_dummy = df["amenity"].str.join(sep='*').str.get_dummies(sep='*')

# add prefix to all amenity dummy columns
df_dummy = df_dummy.add_prefix("amenity_")

# concatenate dummy to the main DataFrame
df = pd.concat([df, df_dummy], axis=1)

In [110]:
# total amenities
df['total_amenties'] = df['amenity'].apply(len)

In [111]:
# remove amenty column since we converted it to dummies or don't want to count
df.drop("amenity", axis=1, inplace=True)

### Extract address and title from rental_title

In [112]:
def extract_city(string):
    city = string.split(",")[-2:-1][0].strip().lower()
    return city

In [113]:
def extract_state(string):
    state = string.split(",")[-1:][0].strip()
    return state

In [114]:
def extract_street(string):
    street = string.split(",")[0].strip()
    return street

In [115]:
def extract_title(string):
    title = string.split(",")[0].strip()
    return title

In [116]:
# change city to lower case
df["city"] = df["city"].transform(lambda x: str(x).lower() if type(x) is str else None)

# extract city from rental_title and merge with existing "city" column
df["city"] = df["city"].combine_first(df["rental_title"].apply(extract_city))

# extract state from rental_title and merge with existing "state" column
df["state"] = df["state"].combine_first(df["rental_title"].apply(extract_state))

# extract street from rental_title and merge with existing "street" column
df["street_address"] = df["street_address"].combine_first(df["rental_title"].apply(extract_street))

# extract rental title from rental_title and replace the original rental_title
df["rental_title"] = df["rental_title"].apply(extract_title)

In [117]:
df['state']=df['state'].apply(lambda x: x.strip())

### School information

In [118]:
def school_rating_count(schools):

    public_school_rating = []
    private_school_count = 0

    school_list = schools.split('}') 
    for i in range(len(school_list)-1):
        rating = school_list[i].split(':')[-1].strip().split('rating')[1].split('gsIcon')[0]
        if rating != 'NR':
            public_school_rating.append(int(rating))
        else:
            private_school_count += 1
            
    if len(public_school_rating) > 0:
        avg_school_rating = round(sum(public_school_rating)/len(public_school_rating),2)
    else:
        avg_school_rating = 999
    return (avg_school_rating, private_school_count)

In [119]:
for i in df.index:
    df.loc[i,'avg_school_rating'],df.loc[i,'private_school_count']  = school_rating_count(df.loc[i,'school'])

### Current cleaned up data

In [121]:
df.head()

,rental_title,borough,street_address,city,state,postal_code,rating,school,bedrooms,bathrooms,...,amenity_tub,amenity_views,amenity_walk-in closets,amenity_washer/dryer,amenity_wheelchair,amenity_wifi,amenity_yoga studio,total_amenties,avg_school_rating,private_school_count
0,Halletts Point,Manhattan,'10 Halletts Pt','astoria','NY','11102',5,"[{'school_name': 'Public Elementary School', '...",0,1.0,...,0,0,0,1,0,1,0,26,4.33,2.0
1,Halletts Point,Manhattan,'10 Halletts Pt','astoria','NY','11102',5,"[{'school_name': 'Public Elementary School', '...",0,1.0,...,0,0,0,1,0,1,0,26,4.33,2.0
2,Halletts Point,Manhattan,'10 Halletts Pt','astoria','NY','11102',5,"[{'school_name': 'Public Elementary School', '...",1,1.0,...,0,0,0,1,0,1,0,26,4.33,2.0
3,Halletts Point,Manhattan,'10 Halletts Pt','astoria','NY','11102',5,"[{'school_name': 'Public Elementary School', '...",1,1.0,...,0,0,0,1,0,1,0,26,4.33,2.0
4,Halletts Point,Manhattan,'10 Halletts Pt','astoria','NY','11102',5,"[{'school_name': 'Public Elementary School', '...",1,1.0,...,0,0,0,1,0,1,0,26,4.33,2.0


In [122]:
df.describe()

,rating,bedrooms,bathrooms,rent,sqft,built_year,apartment_age,property_size,pet_cat,pet_dog,...,amenity_tub,amenity_views,amenity_walk-in closets,amenity_washer/dryer,amenity_wheelchair,amenity_wifi,amenity_yoga studio,total_amenties,avg_school_rating,private_school_count
count,6724.000000,6724.000000,6724.000000,6724.000000,6724.000000,6724.000000,6724.000000,6724.000000,6724.000000,6724.000000,...,6724.000000,6724.000000,6724.000000,6724.000000,6724.000000,6724.000000,6724.000000,6724.000000,6724.000000,6724.000000
mean,394.512641,1.509964,2.741746,2538.930973,671.400506,1604.690958,414.309042,733.225312,0.680547,0.661957,...,0.086109,0.192594,0.134741,0.311273,0.175640,0.283462,0.006395,13.549078,326.835233,1.564396
std,485.441623,1.013249,38.454527,1658.584654,440.572943,488.421971,488.421971,1077.343464,0.466299,0.473078,...,0.280546,0.394366,0.341472,0.463049,0.380541,0.450712,0.079719,10.547068,464.658617,1.401489
min,1.000000,0.000000,0.500000,600.000000,0.000000,999.000000,0.000000,4.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000
25%,5.000000,1.000000,1.000000,1475.000000,475.750000,999.000000,9.000000,295.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,5.000000,0.000000
50%,5.000000,1.000000,1.000000,2225.000000,700.000000,1971.000000,48.000000,571.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,7.000000,2.000000
75%,999.000000,2.000000,1.125000,3150.000000,932.000000,2010.000000,1020.000000,999.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,24.000000,999.000000,2.000000
max,999.000000,7.000000,999.000000,57000.000000,7250.000000,2019.000000,1020.000000,12271.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,31.000000,999.000000,11.000000


In [123]:
df_NY = df.loc[df['state'] == "'NY'"]
df_NJ = df.loc[df['state'] == "'NJ'"]

In [124]:
df_NJ.shape

(214, 88)

In [125]:
df_NY.groupby('borough')['postal_code'].value_counts()

borough        postal_code
Bronx           '10463'       108
                '10468'        52
                '10032'        44
                '10458'        37
                '10462'        37
                '10033'        35
                '10034'        35
                '10031'        34
                '10461'        33
                '10471'        30
                '10453'        29
                '10457'        24
                '10035'        23
                '10467'        23
                '10452'        22
                '10040'        21
                '10456'        19
                '10454'        18
                '10029'        17
                '10455'        16
                '10451'        14
                '10465'        14
                '10459'        12
                '10039'        11
                '10030'        10
                '10466'        10
                '10460'         9
                '10473'         9
                '1003

## Save DataFrame as csv

In [126]:
df.drop(["rental_title",'street_address','school'], axis=1, inplace=True)

In [127]:
# check before save to csv
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6724 entries, 0 to 7796
Data columns (total 85 columns):
borough                              6724 non-null object
city                                 6724 non-null object
state                                6724 non-null object
postal_code                          6724 non-null object
rating                               6724 non-null int64
bedrooms                             6724 non-null int64
bathrooms                            6724 non-null float64
rent                                 6724 non-null float64
sqft                                 6724 non-null int64
built_year                           6724 non-null int64
apartment_age                        6724 non-null int64
property_size                        6724 non-null int64
pet_cat                              6724 non-null int64
pet_dog                              6724 non-null int64
amenity_air conditioning             6724 non-null int64
amenity_alcove                

In [128]:
# save as csv
df.to_csv("data/ny_rental_data_cleaned_v2.csv")

In [ ]:
### list of features we have: 
# clean up zip code; 
# property_info -> unit/size
# school: 
# communication (for next version)
# so basically, we have: postal_code, borough, bedrooms, bathrooms, sqft, rating, amenity, pet_policy, unit/size, school_rating, private_school, 
# df.drop(columns = ['rental_title','neighbor',])

## Full address

In [42]:
# # remove unit from addresses
# df["street_address"] = df["street_address"].apply(lambda x: x.split("Unit")[0].strip())

# # Create full address column
# df["full_address"] = df[['street_address', 'city', 'state']].apply(lambda x: ', '.join(x), axis=1)

## Longitude and latitude

In [49]:
# def geocoder_request(address):
#     return geocoder.google(address,sensor=False)

In [203]:
# longitude = []
# latitude = []

# # loop over all addresses, get longitude, latitude
# for address in df["full_address"]:
   
#     # set maximum retry
#     max_retry = 5
  
#     # attempt to retrieve address by request geocode
#     retry = 0
#     while retry < max_retry:
    
#         # request geocode
#         geocode = geocoder_request(address)
    
#         # pause for a few seconds
#         time.sleep(1)
        
#         # append good longitude and latitude
#         if geocode.status == "OK":
#             longitude.append(geocode.lng)
#             latitude.append(geocode.lat)
           
#             # break out of while loop
#             break
            
#         # count retry
#         retry = retry+1
   
#     # if status code is not okay, print address
#     if not geocode.status == "OK":
#         print("%s status from %s" % (geocode.status, address))
        
#         # place holder
#         longitude.append("")
#         latitude.append("")

# print("Completed collecting geocodes.")
        
# df["longitude"] = longitude
# df["latitude"] = latitude